# Проверка качества

In [1]:
import os
import sys
import time
import json
import pickle
import warnings

from collections import defaultdict

sys.path.append('../')
sys.path.append('old_configs/')
warnings.filterwarnings('ignore')
try:
    sys.modules.pop('src.modeling.autocompletion')
except:
    print('cant do')

import torch
import tokenizers
import numpy as np
import pandas as pd
import torch.nn as nn
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from pygments.token import Token
from catalyst.utils import load_checkpoint, unpack_checkpoint

from src.preprocessing.preprocessing import LexerBasedPreprocessor
from src.preprocessing.tokenization import BertWordPieceTokenizerWrapper, SentencepieceTokenizerWrapper
from src.generation.generation_utils import TokenScoresPostprocessor, NextTokenChooser
from src.generation.autocompletion import AutocompletionModel
from src.generation.postprocessing import get_first_word
from src.preprocessing.tokenization import clever_join
from src.generation.prefix_utils import PrefixMatcher
from src.utils.metrics import reciprocal_rank, relevant_in_k

from run_gpt_ddp_training import GPT2ConfigInitializer

cant do


In [2]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [3]:
# preprocesser
DATA_DIR = '/mnt/data/popov/rcompletion/evaluation_data/'
with open('/home/popov/data/rcompletion_files/december2020_best_model/top_tokens_bigdata_021020.json', 'r') as f:
    top_tokens = json.load(f)
top_tokens = set(top_tokens)
preprocesser = LexerBasedPreprocessor(protected_names=top_tokens)
lexer = preprocesser.lexer

preprocesser = LexerBasedPreprocessor()
lexer = preprocesser.lexer

In [4]:
# tokenizer

base_path = '/mnt/disk/shared/popov/data/rcompletion/bigdata_ver1/'
tokenizer = SentencepieceTokenizerWrapper(f'/mnt/data/porkhun/tokenizer/spm_cased_bpe_16.model')

In [5]:
# score postprocesser
score_postprocesser = TokenScoresPostprocessor(temperature=1)

In [6]:
from gpt_config import Config as gpt_config

initializer = GPT2ConfigInitializer(gpt_config)
model = initializer.init_model()
checkpoint = load_checkpoint('/home/porkhun/model_training/practice/bi_gpt/30_epoch_random_shift/best.pth')
unpack_checkpoint(checkpoint=checkpoint, model=model)

In [7]:
model

BiGPTModel(
  (gpt_left_to_right): GPT2Model(
    (wte): Embedding(16000, 128)
    (wpe): Embedding(512, 128)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [8]:
# model = model.student_model
model = model.eval()

CUDA_DEVICE = 'cuda:0'
model = model.to(CUDA_DEVICE)

In [9]:
def create_df(directory):
    dict_for_df = defaultdict(list)
    with open(directory, 'r') as f:
        for line in f:
            d = json.loads(line)
            for key in d:
                dict_for_df[key].append(d[key])
    return pd.DataFrame(dict_for_df)

In [10]:
if hasattr(model, 'double_context'):
    df = create_df(DATA_DIR+'extracted_events_with_right_context.json')
else:
    df = create_df(DATA_DIR+'extracted_events.json')
df.head()

,url,before_cursor,after_cursor,after_cursor_token,right_context,group,prefix
0,https://api.github.com/repos/CenterForStatisti...,########################\n# ...,otlist,plotlist,"=plotlist,labels=LETTERS[1:4])\ndev.off()",f_key_argument,prefix
1,https://api.github.com/repos/VUW-FAIR/tic-pers...,"setwd(""/Users/mlr/OneDrive - Victoria Universi...",thod,method,"= ""euclid"")\nbarplot(unlist(allDistances))\n\...",f_key_argument,prefix
2,https://api.github.com/repos/jayhesselberth/gg...,"context(""ggsave"")\n\ntest_that(""ggsave creates...",cale,scale,"= 2), c(10, 10))\n})\n\n\n# plot_dev --------...",f_key_argument,prefix
3,https://api.github.com/repos/gtesei/fast-furio...,require(xgboost)\nrequire(methods)\nlibrary(da...,op,drop,"= F]) \n test_set = cbind(test_set , data[...",f_key_argument,prefix
4,https://api.github.com/repos/HadiEO/tropical_r...,require(ggplot2)\nrequire(tidyverse)\nrequire(...,ir,nir,"=seq(0,100,20), swir1=seq(0,100,20), swir2=seq...",f_key_argument,prefix


In [11]:
test_sample = df.iloc[10]['after_cursor_token']
test_sample

'weightsInc'

In [12]:
autocompletion_model = AutocompletionModel(
    preprocessor=preprocesser,
    tokenizer=tokenizer,
    model=model,
    score_postprocesser=TokenScoresPostprocessor(temperature=1.5, penalty_theta=0.5),
    next_token_chooser=NextTokenChooser(do_sample=False),
    max_tokens_amount=5,
    num_beams=5,
    max_num_sequence_return=20,
    input_lines_to_keep=85,
)

Для измерения зависимости от итерации:

In [13]:
lexxerrrr = LexerBasedPreprocessor()

In [14]:
%%time
all_real_outputs = dict()
all_model_outputs = dict()

lines_to_keep = 100
# for lines_to_keep in [None, 85, 70, 80, 100, 140]:
autocompletion_model = AutocompletionModel(
        preprocessor=preprocesser,
        tokenizer=tokenizer,
        model=model,
        score_postprocesser=TokenScoresPostprocessor(temperature=1.5, penalty_theta=0.5),
        next_token_chooser=NextTokenChooser(do_sample=False),
        max_tokens_amount=5,
        num_beams=5,
        max_num_sequence_return=20,
        input_lines_to_keep=lines_to_keep,
    )
    
model_outputs = []
real_outputs = []
bad_indexes = []
for i, elem in tqdm(df.iterrows()):
    if autocompletion_model.double_context:
        test_sample = (elem['before_cursor'], elem['right_context'])
        
    else:
        test_sample = elem['before_cursor']
    try:
        one_model_outputs = autocompletion_model.autocomplete_input(
            test_sample,
            drop_last_word='always' if elem['prefix'] == 'prefix' else 'never',
        )
        one_real_output = elem['after_cursor_token']
        model_outputs.append(one_model_outputs)
        real_outputs.append(one_real_output)
    except:
#         raise
        bad_indexes.append(i)
#     if i > 10:
#         break


CPU times: user 19min 3s, sys: 59.1 s, total: 20min 2s
Wall time: 19min 59s


In [22]:
last_index = 10
list(zip(model_outputs[:-last_index], real_outputs[:-last_index]))

[([], 'plotlist'),
 ([], 'method'),
 ([], 'scale'),
 (['drop', 'names', 'idx', 'cl', 'row_names', 'row_ix', 'row_idx'], 'drop'),
 ([], 'collapse'),
 ([], 'decreasing'),
 ([], 'dependencies'),
 ([], 'tol'),
 ([], 'quote'),
 ([], 'fileext'),
 (['=', 'stdData', 'round', 'my.rank', 'tau', 'divColIndex'], 'na.last'),
 ([], 'id'),
 (['textInput', ',', 'value', 'inputId', 'last'], 'accept'),
 ([], 'delete_dsn'),
 (['y', 'x', 'm'], 'coef'),
 ([], 'labels'),
 ([], 'sep'),
 ([], 'func'),
 ([], 'frequency'),
 ([], 'asValues'),
 ([], 'sep'),
 ([], 'newdata'),
 ([], 'formula'),
 ([], 'fig.show'),
 ([], 'sep'),
 (['scaled',
   'col',
   'lower',
   'x',
   'col.regions',
   'lower.panel',
   'lower.tail',
   'col.names',
   'col.cex',
   'col.center',
   'lower.level',
   'lower.size'],
  'log'),
 ([], 'cluster'),
 ([], 'replace'),
 (['col',
   'size',
   'color',
   'block',
   'block_size',
   'pointsize',
   'blocksize',
   'point',
   'block.size',
   'block.color',
   'block.col',
   'blocksize

In [14]:
len(bad_indexes)

4

In [18]:
# delete all brackets from left context
real_o = real_outputs
model_o = model_outputs
relevances = [
    [int(x == one_r_o) for x in one_model_o]
    for one_r_o, one_model_o in zip(real_o, model_o)
]
key_metrics = [
    [relevant_in_k(one_r, k=k) for k in range(1, 6)] + [reciprocal_rank(one_r)]
    if one_r else [0] * 6
    for one_r in relevances
]
key_metrics = np.array(key_metrics).mean(axis=0)
key_metrics

array([0.15558647, 0.20760726, 0.24117733, 0.26222727, 0.2764314 ,
       0.20334682])

In [19]:
# output with bug tokenize
real_o = real_outputs
model_o = model_outputs
relevances = [
    [int(x == one_r_o) for x in one_model_o]
    for one_r_o, one_model_o in zip(real_o, model_o)
]
key_metrics = [
    [relevant_in_k(one_r, k=k) for k in range(1, 6)] + [reciprocal_rank(one_r)]
    if one_r else [0] * 6
    for one_r in relevances
]
key_metrics = np.array(key_metrics).mean(axis=0)
key_metrics

array([0.05373704, 0.0851992 , 0.10527836, 0.1165747 , 0.12558865,
       0.08251272])

In [15]:
# output with bugfix tokenize
real_o = real_outputs
model_o = model_outputs
relevances = [
    [int(x == one_r_o) for x in one_model_o]
    for one_r_o, one_model_o in zip(real_o, model_o)
]
key_metrics = [
    [relevant_in_k(one_r, k=k) for k in range(1, 6)] + [reciprocal_rank(one_r)]
    if one_r else [0] * 6
    for one_r in relevances
]
key_metrics = np.array(key_metrics).mean(axis=0)
key_metrics

array([0.15558647, 0.20760726, 0.24117733, 0.26222727, 0.2764314 ,
       0.20334682])

In [18]:
key_to_metrics = dict()

for one_key in sorted(all_real_outputs.keys()):
    real_o = all_real_outputs[one_key]
    model_o = all_model_outputs[one_key]
    
    relevances = [
        [int(x == one_r_o) for x in one_model_o]
        for one_r_o, one_model_o in zip(real_o, model_o)
    ]
    
    key_metrics = [
        [relevant_in_k(one_r, k=k) for k in range(1, 6)] + [reciprocal_rank(one_r)]
        if one_r else [0] * 6
        for one_r in relevances
    ]
    
    key_metrics = np.array(key_metrics).mean(axis=0)
    key_to_metrics[one_key] = key_metrics

In [19]:
key_metric_list = sorted(key_to_metrics.items())

In [20]:
key_metric_list

[]

In [21]:
key_metrics

nan

In [22]:
fig, ax = plt.subplots(2, 3)

fig.set_figwidth(17)
fig.set_figheight(7)

plt.subplot(2, 3, 1)
plt.plot([x[0] for x in key_metric_list[:-1]], [x[1][-1] for x in key_metric_list[:-1]], '-o')
plt.plot([x[0] for x in key_metric_list[:-1]], [key_metric_list[-1][1][-1]] * (len(key_metric_list) - 1), '--')


plt.legend(['distilled model'], fontsize=14)
plt.ylabel('MPR')
plt.xlabel('keeped line amount')
plt.xticks([x[0] for x in key_metric_list[:-1]])
    

for top_i in range(0, 5):
    plt.subplot(2, 3, top_i + 2)    
    plt.plot([x[0] for x in key_metric_list[:-1]], [x[1][top_i] for x in key_metric_list[:-1]], '-o')
    plt.plot([x[0] for x in key_metric_list[:-1]], [key_metric_list[-1][1][top_i]] * (len(key_metric_list) - 1), '--')

    plt.xlabel('keeped line amount')
    plt.xticks([x[0] for x in key_metric_list[:-1]])
    plt.ylabel(f'right in top_{top_i + 1}')
plt.show()    

IndexError: list index out of range

In [15]:
x = torch.tensor([[   13,   180,   109,   868, 15920,     4,   148,  2191,    59,     5,
            13,   180,   109,   599,    57, 15930,   417,   180, 15928, 15930,
           417,  2191, 15928, 15930,   417,  1066,    59,     5,   728,     5,
            19,  1322, 15917,  1440,    92,     8,  1897,  3975, 15920,   308,
           553,     5,    19,     8,  1987,  3975, 15920,   308,   553,     5,
            19,   310, 15920, 15924,    15, 15940,  4760,     5,   835,     8,
          2015,  1667, 15920,   465,  9831, 15920,     4,  1759, 15924,  6692,
          2496, 15917,  2418, 15920,     4,  1759, 15924,   268, 12437,     8,
          1277, 15924,  2717,     4,  3235,     5,    62,     8,  1412, 15924,
          9497,     5,  1156,     4,  3235,     5,    62,   106,  3013,    57,
           507,    59,     5,   728,     5,    13,  1104,    59,     5,   728,
             5,    19,   382, 15920, 10418, 15928, 15930,   501,   382, 15920,
            73, 15924, 15928, 15930,   501,   382, 15920,   965, 15928, 15930,
           501,   310, 15920, 15924,    15, 15940,  4760,     5,   835,     8,
          2163, 15972,  2796, 15920,   695,    48,     5,    69,     5,    69,
             5,   728,     5,   107,  1367, 15928,     4,  1277, 15924,   220,
           870, 15928,     4,  1412, 15924,   268, 10945, 15920,     4,   362,
             4,  1759, 15924,  6692,  2496, 15917,  2418, 15920,     4,  1759,
         15924,   268,   220,  1140,   102,  1175,   102,   925, 15928,   402,
            57, 15924, 15919,  1044,  2899,  1066, 15917,    80,    59,     5,
            13,  1066, 15917,   571,    59,     5,    13,  1066, 15917,   925,
            59,     5,    13,  1066, 15917,   323,    59,     5,    13,  1066,
            59,     5,    19,  2725, 15920,  2055,    59,     5,    13,   166,
           102,   205,    59,     5,    13,  1066,    59,     5,    19,   191,
         15920, 15924, 15949,    15, 15949, 15930, 14893, 15931,   166, 15920,
          2055,    59,     5,    13,   166,   102,   205,    59,     5,    13,
          1066,    59,     5,  4515,  3864, 15920,     4,  1759, 15924,  6692,
          2496, 15917,  2418, 15920,     4,  1759, 15924,   268,   220,  5240,
         15928,   814, 15920, 15935, 15921, 15972,  2796, 15920, 15935, 15919,
          1367, 15928,     4,  1277, 15924,   220,   870, 15928,     4,  1412,
         15924,  1434,   106,  1322, 15917,  1440,    92,     8,  2310,   327,
            92,   310, 15920, 15924,    15, 15940,  4760,     5,   835,     8,
          2852,     4,  1759, 15924,  6692,  2496, 15917,  2418, 15920,     4,
          1759, 15924,  7314,     8,  3036,  2194, 15920,     4,   682, 15930,
           501,     8,  3115, 15972,  2796, 15920, 15930,   417,  1367, 15928,
             4,  1277, 15924,   220,   870, 15928,     4,  1412, 15924,   268,
             8,  3262,   939, 15920,     4,   993, 15984,   939, 15920,     4,
          1054,     8,  3447,     4,  4172,     5,  4056,     4,   178, 15982,
             4,  4405,     5,    62,     8,  3572,    45, 15917,   211, 15920,
             4,   920,     8,  7682, 15924,  6846,   554, 15920,    45, 15928,
             4,   887,   212, 15920,   643, 15928,     4,   123,   848,   485,
         15938,  4537, 15920,  5240, 15928, 15972,  2796, 15919, 15934,  1574,
         15928,   327, 15920,  1367, 15928,     4,  1277, 15924,   220,   870,
         15928,     4,  1412, 15924,  1642,   274,    59,     5,   728,     5,
           979,  1140,  1155,  1175,  1155,   219, 15938,   778,   923,   971,
         15938,  2250, 15920,  2964, 15928,     4,   808,   820, 15928,     4,
          1147, 15954,   219, 15920,   571, 15917,   183, 15928,   734, 15938,
           166, 15920,   274,    59,     5,   107,   571, 15917,   166, 15928,
           734, 15938,   166, 15920,   274,    59,     5,   116,   106,  3013,
            57,   507,    59,     5,   728,     5,    13,  1104,    59,     5,
           728,     5,    19,   174, 15938,   498, 15920]])

In [17]:
y = torch.tensor([1, 0, 0, 1, 1, 0, 0, 1, 0, 1]).long()

In [20]:
x[y]

IndexError: index 1 is out of bounds for dimension 0 with size 1